In [1]:
# 필요패키지 import
import time
from tqdm.notebook import tqdm
import json
import urllib
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyautogui
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import clipboard
import pyperclip
import dataframe_image
from google_play_scraper import Sort, app, reviews_all
import nltk
import folium
from selenium.webdriver.common.action_chains import ActionChains

# konlpy
from konlpy.tag import Okt
okt=Okt()

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 셀레니움 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC

# webdriver 자동설정
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(), options=chrome_options)
    return driver
driver=set_chrome_driver()
driver.close()


import time
import warnings
warnings.filterwarnings(action='ignore')
from bs4 import BeautifulSoup

from pytesseract import Output
import pytesseract
import cv2

import urllib.request
from PIL import Image
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"
    
urllib._urlopener = AppURLopener()

import os
import keyboard
import getpass
from sqlalchemy import create_engine

def driverxpath(x):
    return driver.find_element(By.XPATH, x)

def drivercss(x):
    return driver.find_element(By.CSS_SELECTOR, x)

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
        else:
            print(f'{directory} 폴더가 이미 존재합니다.')
    except OSError:
        print ('Error: Creating directory. ' +  directory)

# 네이버 지도 음식점 리뷰 크롤링
-- 엄청 오래 걸리고 한번 됐다고 해서 코드를 고쳐야 할 수도 이

In [142]:
# 검색어 입력받기
# search=input('검색어를 입력하세요. : ')

# 드라이버 열기
url=f'http://map.naver.com/p/search/{search}'
driver=set_chrome_driver()
driver.get(url)
driver.maximize_window()

time.sleep(3)

# searchIframe으로 이동
driver.switch_to.default_content()
driver.switch_to.frame('searchIframe')

restaurants=[]

for i in range(1,3):
    # 식당 클릭
    driverxpath(f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a/div/div/span[1]').click()
    
    time.sleep(3)
    # entryIframe으로 이동
    driver.switch_to.default_content()
    driver.switch_to.frame('entryIframe')
    
    time.sleep(3)
    # 식당 상호
    restaurant_name = driverxpath('//*[@id="_title"]/div/span[1]').text
    
    try:
        # 식당 주소
        addr = driverxpath('//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/a/span[1]').text
    except:
        addr=None
        pass

    # 리뷰 탭 클릭
    for i in range(1,10):
        if driverxpath(f'//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[{i}]/span').text == '리뷰':
            driverxpath(f'//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[{i}]/span').click()
            break
    
    time.sleep(3)
    # html 추출
    html=BeautifulSoup(driver.page_source)
    
    try:
        # 별점 span.xobxM.fNnpD
        point=html.select('span.xobxM.fNnpD')[0].text
    except:
        point = None
        pass

    reviews=[]
    
    time.sleep(2)

    for i in range(5):
        # 유저 span.pui__NMi-Dp
        user = html.select('span.pui__NMi-Dp')[i].text
        # 메뉴 span.pui__ETqMYH
        menu = html.select('span.pui__ETqMYH')[i].text
        # 개요 div.pui__-0Ter1
        summ = html.select('div.pui__-0Ter1')[i].text
        # 리뷰 내용 a.pui__xtsQN-
        content = html.select('a.pui__xtsQN-')[i].text
        # 태그 div.pui__HLNvmI
        tag = html.select('div.pui__HLNvmI')[i].text.split('+')[0]

        reviews.append(
            {
                'user':user
                , 'menu':menu
                , 'summ':summ
                , 'content':content
                , 'tag':tag
            }
        )
    
    restaurants.append(
        {
            'restaurant_name':restaurant_name
            , 'address':addr
            , 'point':point
            , 'reviews':pd.DataFrame(reviews).to_dict()
        }
    )
    
    driverxpath('//*[@id="app-root"]/div/div/header/a[2]').click()

    driver.switch_to.default_content()
    driver.switch_to.frame('searchIframe')
    
driver.close()